# Импорт данных и библиотек

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
# Чтение файла
data_path = '../materials/data/Iris.csv'
iris_data = pd.read_csv(data_path)

In [ ]:
iris_data

# Восстановление и очистка данных

In [ ]:
# Проверка наличия пропущенных данных
missing_data = iris_data.isnull().sum()
print(missing_data)

В данном случае, все столбцы - (Id, SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm, Species) не содержат пропусков, и данные готовы для дальнейшей обработки.

Пример что делать если пропущенные данные всё же есть:
Например, можно использовать среднее значение для заполнения пропусков:

In [ ]:
iris_data.fillna(iris_data.mean(), inplace=True)

# Проверка, что пропущенных данных больше нет
missing_data_after = iris_data.isnull().sum()

# Вывод информации о данных после очистки
iris_data.info()

# Проверка распределения данных

In [ ]:
# Гистограммы распределения числовых параметров
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

sns.histplot(iris_data['SepalLengthCm'], kde=True, ax=axes[0, 0], bins=30, color='skyblue')
axes[0, 0].set_title('Sepal Length Distribution')

sns.histplot(iris_data['SepalWidthCm'], kde=True, ax=axes[0, 1],bins=30, color='salmon')
axes[0, 1].set_title('Sepal Width Distribution')

sns.histplot(iris_data['PetalLengthCm'], kde=True, ax=axes[1, 0],bins=30, color='lightgreen')
axes[1, 0].set_title('Petal Length Distribution')

sns.histplot(iris_data['PetalWidthCm'], kde=True, ax=axes[1, 1],bins=30, color='gold')
axes[1, 1].set_title('Petal Width Distribution')

plt.tight_layout()
plt.show()

# Ящиковые диаграммы распределения по классам
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))

for i, column in enumerate(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']):
    sns.boxplot(x='Species', y=column, data=iris_data, ax=axes[i])

plt.tight_layout()
plt.show()

Очевидно не нормальное распределение

In [ ]:
# матрица диаграмм рассеяния
plt.figure(figsize=(15,15))
sns.pairplot(iris_data, hue='Species', diag_kind='kde')
plt.show()

  Взглянув на график, мы можем увидеть, что, похоже, измерения чашелистиков и лепестков позволяют относительно хорошо разделить три класса. Это означает, что модель машинного обучения, вероятно, сможет научиться разделять их.

# Проведение EDA и визуализация отобранных параметров (влияющих на целевую переменную)

In [ ]:
# Визуализация матрицы корреляции с тепловой картой
plt.figure(figsize=(10, 5))
sns.heatmap(iris_data.corr(),annot=True)
plt.show()

Все переменные жестко влияют

п.с. это ирисы тут всегда так

# Разделение набора данных на обучающую и тестовую выборку

In [35]:
# Разделение датасета на признаки(Х) и целевую переменную(У)
X = iris_data.drop('Species', axis=1)
Y = iris_data['Species']

In [43]:
# Разделение датасета на обучающую и тестовую выборку (80% - обучение, 20% - тесты)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=42)

In [ ]:
# Вывод размеров полученных выборок
print("Размер обучающей выборки:", X_train.shape, Y_train.shape)
print("Размер тестовой выборки:", X_test.shape, Y_test.shape)

# Выдвижение гипотезы о возможных применимых алгоритмах машинного обучения

Логистическая регрессия:

 • Гипотеза: Логистическая регрессия хорошо подходит для бинарной и многоклассовой классификации. Учитывая, что у нас три класса (iris setosa, iris versicolor, iris virginica), логистическая регрессия может быть эффективным методом, особенно если классы хорошо разделимы.

 • Описание: Логистическая регрессия используется для моделирования вероятности принадлежности к классу. Она основывается на логистической функции, которая преобразует комбинацию признаков в вероятность.

Дерево решений:

• Гипотеза: Деревья решений могут хорошо справляться с нелинейными зависимостями между признаками и целевой переменной. В данной задаче, где параметры могут варьироваться в разных направлениях, деревья решений могут быть гибким и мощным инструментом.

• Описание: Дерево решений строит структуру дерева, где каждый узел представляет собой тест на одном из признаков, а каждый лист - прогнозный класс.

K-ближайших соседей (KNN):

• Гипотеза: KNN может быть эффективным в задачах, где объекты одного класса склонны сгруппироваться в пространстве. В данной задаче, если схожие ирисы образуют группы, KNN может хорошо справляться с классификацией.

• Описание: KNN классифицирует объекты на основе их близости к соседним объектам. Когда новый объект появляется, он присваивается класс, который наиболее часто встречается среди k ближайших соседей.

# Обучение выбранных моделей (библиотека sklearn)


## Обучение модели с помощью агоритма логистической регрессии

In [ ]:
# Создание и обучение модели логистической регрессии
logistic_regression_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_model.fit(X_train, Y_train)

In [48]:
# Предсказание на тестовой выборке
y_pred_lr = logistic_regression_model.predict(X_test)

In [53]:
# Оценка производительности модели
accuracy_lr = accuracy_score(Y_test, y_pred_lr)
conf_matrix_lr = confusion_matrix(Y_test, y_pred_lr)
classification_report_lr = classification_report(Y_test, y_pred_lr)

In [ ]:
# Вывод результатов
print("Accuracy логистической регрессии:", accuracy_lr)
print("\nConfusion Matrix логистической регрессии:\n", conf_matrix_lr)
print("\nClassification Report логистической регрессии:\n", classification_report_lr)

Анализ Confusion matrix для метода логистической регресии:


• Все элементы на главной диагонали (10, 10, 10) - это True Positives, что означает, что все объекты каждого класса были верно классифицированы.

• Нет False Positives (0 вне главной диагонали) и False Negatives (0 вне главной диагонали), что подтверждает отсутствие ошибок классификации.

Анализ Classification Report для метода логистической регресии:


• Precision (точность), Recall (полнота), и F1-score для каждого класса равны 1.00. Это указывает на то, что модель логистической регрессии идеально справилась с классификацией.

 • Accuracy (точность) также равна 1.00, что означает, что все предсказания модели верны.

В целом, вывод свидетельствует о том, что модель логистической регрессии имеет отличную производительность на тестовой выборке, справляясь с классификацией всех трех классов ирисов с высокой точностью и полнотой.

## Обучение модели с помощью агоритма k-ближайших соседей

In [ ]:
# Создание и обучение модели k-ближайших соседей
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, Y_train)

In [59]:
# Предсказание на тестовой выборке
y_pred_knn = knn_model.predict(X_test)

In [62]:
# Оценка производительности модели
accuracy_knn = accuracy_score(Y_test, y_pred_knn)
conf_matrix_knn = confusion_matrix(Y_test, y_pred_knn)
classification_report_knn = classification_report(Y_test, y_pred_knn)

In [ ]:
# Вывод результатов
print("Accuracy k-ближайших соседей:", accuracy_knn)
print("\nConfusion Matrix k-ближайших соседей:\n", conf_matrix_knn)
print("\nClassification Report k-ближайших соседей:\n", classification_report_knn)

Анализ Confusion matrix для метода k-ближайших соседей:

• Все элементы на главной диагонали (10, 10, 10) - это True Positives, что
означает, что все объекты каждого класса были верно классифицированы.

 • Нет False Positives (0 вне главной диагонали) и False Negatives (0 вне главной диагонали), что подтверждает отсутствие ошибок классификации.

Анализ Classification Report для метода k-ближайших соседей:

• Precision, Recall и F1-score для каждого класса равны 1.00. Это указывает на то, что модель k-ближайших соседей идеально справилась с классификацией.

 • Взвешенное среднее также равно 1.00, что подчеркивает общую точность модели.

В целом, результаты для модели k-ближайших соседей также указывают на высокую точность и отсутствие ошибок классификации на данном наборе данных.

## Обучение модели с помощью Дерева принятия решений

In [ ]:
# Создание и обучение модели дерева решений
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, Y_train)

In [66]:
# Предсказание на тестовой выборке
y_pred_dt = decision_tree_model.predict(X_test)

In [68]:
# Оценка производительности модели
accuracy_dt = accuracy_score(Y_test, y_pred_dt)
conf_matrix_dt = confusion_matrix(Y_test, y_pred_dt)
classification_report_dt = classification_report(Y_test, y_pred_dt)

In [ ]:
# Вывод результатов
print("Accuracy дерева решений:", accuracy_dt)
print("\nConfusion Matrix дерева решений:\n", conf_matrix_dt)
print("\nClassification Report дерева решений:\n", classification_report_dt)

Анализ Confusion Matrix для дерева решений:

• Все элементы на главной диагонали (10, 10, 10) - это True Positives, что означает, что все объекты каждого класса были верно классифицированы.

 • Нет False Positives (0 вне главной диагонали) и False Negatives (0 вне главной диагонали), что подтверждает отсутствие ошибок классификации.

Анализ Classification Report для дерева решений:

• Precision, Recall и F1-score для каждого класса равны 1.00. Это указывает на то, что модель дерева решений идеально справилась с классификацией.

 • Взвешенное среднее также равно 1.00, что подчеркивает общую точность модели.

Результаты для модели дерева решений также свидетельствуют о высокой точности и отсутствии ошибок классификации на данном наборе данных. В целом, все три модели показывают идеальные результаты на предоставленных данных Iris.

### Метрики MAE (Mean Absolute Error) и MSE (Mean Squared Error) обычно используются в задачах регрессии, а не в задачах классификации.

# Выбор наилучшей модели на основе вычисленных метрик


В ходе анализа и обучения трех различных моделей машинного обучения – логистической регрессии, k-ближайших соседей и дерева решений, на наборе данных ирисов – были получены высокие показатели производительности. Все три модели достигли максимальной точности (Accuracy) в 1.0, что свидетельствует о безошибочной классификации на тестовой выборке.

Подробный анализ метрик, таких как Precision, Recall и F1-score для каждого класса, а также матрицы ошибок (Confusion Matrix), подтвердил отсутствие ложных срабатываний и упущений для всех классов. Это говорит о высокой надежности всех трех моделей в различении классов ирисов (Iris-setosa, Iris-versicolor, Iris-virginica).

Таким образом, на основании анализа метрик и общей точности, все три модели оказались идеальными для данной задачи классификации. Однако, учитывая простоту и интерпретируемость, логистическая регрессия может быть предпочтительной в реальных условиях, если обеспечивает достаточную производительность. В конечном итоге, выбор модели может зависеть от специфики бизнес-задачи, требований к интерпретируемости и ресурсных ограничений.